In [1]:
# Load env variables and create client
import base64
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-sonnet-4-5"

In [2]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [3]:
# TODO: Read image data, feed into Claude
with open("pdf/earth.pdf", "rb") as image_file:
    document_data = base64.b64encode(image_file.read()).decode("utf-8")
    
messages = []

add_user_message(messages, [
    {
        "type": "document",
        "source": {
            "type": "base64",
            "media_type": "application/pdf",
            "data": document_data
        }
    },
    {
        "type": "text",
        "text": "Please summarize the key points from the attached PDF document."
    }
])

chat(messages)

Message(id='msg_01GSTqyjfUgcCrj3YWY6N8H2', content=[TextBlock(citations=None, text='# Summary of Earth Wikipedia Article\n\n## Basic Overview\n- **Earth** is the third planet from the Sun and the only known astronomical object to harbor life\n- It\'s an ocean world with 70.8% of its surface covered by water\n- The remaining 29.2% is land, mostly continental landmasses\n\n## Physical Characteristics\n- **Mean radius**: 6,371.0 km\n- **Mass**: 5.972 × 10²⁴ kg\n- **Densest planet** in the Solar System and largest of the rocky planets\n- **Circumference**: ~40,000 km (25,000 miles)\n- **Distance from Sun**: ~8 light-minutes away\n- **Orbital period**: 365.25 days (one year)\n- **Rotation period**: ~23 hours 56 minutes\n\n## Key Features\n- **Atmosphere**: Primarily nitrogen (78%) and oxygen (21%), with water vapor and other gases\n- **Average surface temperature**: 14.76°C (58.57°F)\n- **Magnetosphere**: Generated by liquid outer core, deflects solar winds and cosmic radiation\n- **Moon**: